### Clifford basedなHamiltonian簡略化アルゴリズムを作りたい。　　
構成としては以下のような感じでいいと思う　　
1. Hamiltonianを表すクラスとしては、non-zeroなHamiltonianのPauli vecが格納されたlis_vecと  
対応するampが格納されたlis_ampを用意する。
2. あらかじめ、cli_rot(vec1, vec2)という関数を作っておく。これはe^{-i pi/2 vec2 }(vec1)e^{i pi/2 vec2 }  
というClifford操作を表すための関数で、その内部操作としては、vec1がvec2と交換する場合は変化なし、交換しない場合はpauli string  
を足したものが返ってくると言った感じになる。←そのために、{0123}^2→{0123}で掛け算に対応するpauliを作っておきたい。  
注：cost functionを評価する上ではamplitudeにくっつく1,-1,i,-iなどの係数は無視してもいい(どうせcliffordなので  
重ね合わせで消えるみたいなこともない)のだが、何となく関数の返り値を完璧にしたいので、ちゃんと係数も考えることにする  
3. 具体的には、全体の最適化を行う関数optimize_cliffordの返り値は ①最適な基底を与えるClifford回転、具体的には、  
list[list]で、外側のlistのindex+1(1,2,...)はcircuitの何番目の深さか、内側のlistは対応する深さでのcliffordのlist  
(具体的には、これ自身も[[(1,14), (3,3)], [(4,11), (2,1)], ...]、みたいな形式の[(qubitのペア), (XYZの組み合わせ)]  
という2qubit cliffordの組み合わせを指定するものになっている。できることならqiskitと対応させたいものだが...時間的になあ...)  
②その結果のlis_vecとlis_amp(実はlis_ampも1,-1, -i, i倍を受ける可能性が微妙にあるので、これもoutputに含める)  
の2つになるようにしたい。また、optimize_cliffordの引数はlis_vec, lis_amp, cost_func, max_iter, min_cost_decの5つ  
でいいだろう。  
4. optimize_cliffordのコードの流れとしては、以下のようなものを考えている:  
①lis_vecの長さからqubit数nを取得、lis_ampと同じかを確かめる  
②for文をmax_iterについて作成、rest_qubit=(1,...,n)を作る  
③min_ind=0 ((n,n,4,4)型のindを入れるもの、後述のcostを最小化するもの)とmin_val=1を作る。  
④(i,k, si, sk)要素(i<k)についてfor文、i番目がsi、k番目がsk、それ以外が0な長さnのpauli_word　vec_rotを作る  
⑤for文の中で、Σ_{vec1\in lis_vec} [cost(vec1)-cost(cli_rot(vec1, vec_rot))]*abs(lis_amp of vec1) を計算、  
結果がmin_valより小さければmin_indを更新、min_valも更新  
⑥for文終了後、もしmin_val<0ならmin_indを元にvec1=cli_rot(vec1, vec_rot)  
⑦rest_qubitからi,jを取り除く。  

↑ただ、ここでそもそもvecのclassとしてlen=nのものではなくlis_qとlis_pの組を指定したほうがいいかも

In [ ]:
import numpy as np
# lis_q:qubit番号、lis_p:pauli番号
def nonzero2vec(lis_q, lis_p, n):
    vec = np.zeros(n)
    for i in range(len(lis_q)):
        vec[lis_q[i]] = lis_p[i]

# cli_rot, cost,
def cost(vec):
    